####  import related modules

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import random

#### File finder

In [2]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add




#### Folder creator

In [3]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

path="iot-data"

#### Delete unnecessary files

In [28]:
#### Delete unnecessary files
def killthemall(path,key):
    them=find_the_way(path,key)
    for t in them:
        try:
            os.remove(t)
        except:
            print(f"error about delete {t} file")
#killthemall(path,".csv")

In [45]:
# find pcap file in path folder

pcaps=find_the_way(path,".pcap")
pcaps

['iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_17_23_20.68s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_18_38_57.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_19_39_01.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_32_13.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_51_27.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_13_35.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_35_48.64s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_57_55.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_20_02.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_42_11.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_17_04_20.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\

#### If your computer has difficulty processing large pcap files, you can split them using the following script

In [30]:
#for i in pcaps:
    #if os.path.getsize(i)//(1024*1024) >500:
        #temp=i.replace("\\","/")
        #command=f"editcap  -c 1000000 {temp} {temp.replace("pcaps/","pcaps/smaller_")}"
        #os.system(command)
        #print(f"{i} made smaller")
        #os.remove(temp)

In [31]:
### Renaming tools

In [32]:
# Feature list
feature=["eth.src",
    #'dstport',
  #'dstport_class',
  'http.chat',
  'http.notification',
  'http.request.method',
  'ip.flags.df',
  'ip.len',
  'ip.proto',
  'ip.ttl','ipv6.hopopts.nxt',
  #'srcport',
  #'srcport_class',
  'tcp.ack',
  'tcp.analysis.ack_rtt',
  'tcp.analysis.bytes_in_flight',
  'tcp.analysis.initial_rtt',
  'tcp.analysis.push_bytes_sent',
  'tcp.completeness',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.push',
  'tcp.flags.str',
  'tcp.flags.syn',
  'tcp.hdr_len',
  'tcp.len',
  'tcp.nxtseq',
  'tcp.srcport',
  'tcp.stream',
  'tcp.time_delta',
  'tcp.time_relative',
  'tcp.window_size',
  'tcp.window_size_scalefactor',
  'tcp.window_size_value',
  'tls.record.length',
  'udp.checksum.status',
  'udp.dstport',
  'udp.srcport',
  'udp.time_delta',
  'udp.time_relative',"_ws.col.Protocol"]


# dictionary to convert string to category
strs={"http_request_method": {'DELETE': 1, 'GET': 2, 'M-SEARCH': 3, 'NOTIFY': 4, 'POST': 5, 'PUT': 6,'SUBSCRIBE':7, 'UNSUBSCRIBE':8,"HEAD":9},
"tcp_flags_str": {'·······AP··F': 1, '·······AP···': 2, '·······A·R··': 3, '·······A··S·': 4,
    '·······A···F': 5, '·······A····': 6, '·········R··': 7,  '··········S·': 8,'····CE····S·':9, "····C··AP···":10,"···········F":11}}

# string to category feature list
str_cols=["http.request.method",
"tcp.flags.str"]

# MAC address of devices and their names. It will use for labelling
macs=MAC_list={"f8:da:0c:7b:0a:8f":"labelprinter","70:2c:1f:3b:36:53":"fridge","18:b4:30:c8:d8:28":"nest-tstat","18:74:2e:41:4d:35":"echodot","38:8c:50:68:d7:5c":"lgtv-wired","50:c7:bf:5a:2e:a0":"tplink-plug","68:27:37:52:31:06":"samsungtv-wifi","84:c0:ef:2f:42:cc":"samsungtv-wired","00:fc:5c:e0:81:86":"microseven-camera","c0:97:27:6c:ae:42":"washer-old","c0:97:27:81:67:99":"washer","c0:97:27:73:aa:38":"dryer","b4:e6:2a:27:34:b7":"dishwasher","f0:db:e2:f2:79:2e":"iphone","6c:72:20:c5:0a:3f":"dlink-mov","24:fd:5b:04:1b:75":"smartthings-hub","98:84:e3:e4:35:bd":"ring-doorbell","00:0e:f3:3b:85:e5":"insteon-hub","88:de:a9:08:03:b9":"roku-tv","34:ce:00:d6:91:e2":"xiaomi-cam1","34:ce:00:d6:8f:c1":"xiaomi-cam2","ec:fa:bc:82:20:bb":"bulb1","34:ce:00:b1:2c:5e":"xiaomi-movingcam1","34:ce:00:b0:1f:fb":"xiaomi-movingcam2","24:fd:5b:02:1d:3a":"t-smartthings-hub","00:17:88:68:5f:61":"t-philips-hub","14:91:82:b4:4b:5f":"t-wemo-plug","dc:ef:ca:22:b8:ff":"wink-hub2-wifi","00:21:cc:4d:ce:8c":"wink-hub2","50:c7:bf:a0:f3:76":"tplink-bulb","00:71:47:c0:91:93":"echospot","dc:4f:22:c1:58:05":"magichome-strip","84:18:26:7d:cf:a2":"lightify-hub","20:df:b9:5f:41:7e":"google-home-mini","fc:65:de:5f:15:0a":"t-echodot","34:ce:00:83:99:35":"xiaomi-hub","34:ce:00:99:9b:83":"philips-bulb","6c:56:97:35:39:f4":"firetv","84:8e:0c:71:08:70":"iphone-daniel","0c:2a:69:0e:91:16":"ikettle","e4:e0:a6:3c:3b:d1":"ipad","bc:75:74:39:96:9e":"nexus6p2","b4:e6:2a:8c:a2:c2":"lgtv-wifi","64:bc:0c:2c:5a:54":"nexus5x1","64:bc:0c:2c:5a:af":"nexus5x2","64:bc:0c:2c:5a:42":"nexus5x3","64:bc:0c:6a:81:f3":"nexus5x4","64:bc:0c:2c:59:b9":"nexus5x5","9c:8e:cd:0a:33:1b":"amcrest-cam-wired","34:ce:00:8b:22:74":"xiaomi-strip","20:f8:5e:cc:18:1f":"brewer","08:66:98:a2:21:9e":"appletv","f8:cf:c5:c9:38:f0":"nexus6","9c:8e:cd:0a:33:5f":"amcrest-cam-wifi","fc:a1:83:38:e0:2d":"echoplus","b0:ce:18:27:9f:e4":"sengled-hub","b0:fc:0d:c9:00:4c":"cloudcam","b0:d5:9d:b9:f0:b4":"yi-camera","78:a5:dd:1a:15:19":"wansview-cam-wired","ec:3d:fd:34:b4:1b":"wansview-cam-wifi","00:03:7f:4f:c6:b5":"blink-security-hub","f4:b8:5e:31:73:db":"blink-camera","00:0c:43:20:32:bb":"luohe-spycam","ae:ca:06:08:d3:e6":"lefun-cam-wired","00:0c:43:27:4f:c1":"lefun-cam-wifi","d4:e6:b7:43:d5:e7":"galaxytab-a","d8:f7:10:c3:34:e4":"invoke","dc:4f:22:28:b6:5b":"sousvide","d8:28:c9:10:b5:60":"microwave","7c:c7:09:56:6e:48":"zmodo-doorbell","7c:49:eb:35:7a:49":"xiaomi-ricecooker","d4:61:9d:ed:fb:c3":"iphone7-pink","88:6b:6e:4c:12:9b":"iphone7-gray","64:bc:0c:80:7e:1f":"nexus5x3","64:1c:ae:50:57:2a":"samsungtv-wifi","fc:03:9f:93:22:62":"samsungtv-wired","d0:52:a8:a4:e6:46":"smartthings-hub","c8:d0:83:21:3d:4b":"iphone","20:df:b9:13:e5:2e":"google-home-mini","54:60:09:6f:32:84":"google-home","78:a5:dd:28:a1:b7":"wansview-cam-wired","0c:8c:24:0b:be:fb":"yi-camera","00:03:7f:96:d8:ec":"blink-security-hub","f4:b8:5e:68:8f:35":"blink-camera","70:ee:50:36:98:da":"netatmo-weather-station","b0:c5:54:12:33:40":"dlink-camera","58:ef:68:99:7d:ed":"t-wemo-plug","00:0e:f3:2c:d4:04":"insteon-hub","ae:ca:06:0e:ec:89":"bosiwo-camera-wired","00:0c:43:03:51:be":"bosiwo-camera-wifi","ec:b5:fa:00:98:da":"t-philips-hub","c8:3a:6b:fa:1c:00":"roku-tv","84:18:26:7c:1a:56":"lightify-hub","50:c7:bf:ca:3f:9d":"tplink-bulb","b0:ce:18:20:43:bf":"sengled-hub","68:c6:3a:ba:c2:6b":"sousvide","cc:f7:35:49:f4:05":"echodot","5c:41:5a:29:ad:97":"echospot","00:fc:8b:84:22:10":"echoplus","b0:f1:ec:d4:26:ae":"allure-speaker","50:c7:bf:b1:d2:78":"tplink-plug","b0:be:76:be:f2:aa":"tplink-plug2","fc:ee:e6:2e:23:a3":"charger-camera","78:11:dc:76:a7:08":"xiaomi-cam1","78:11:dc:76:69:b0":"xiaomi-cam2","cc:f7:35:25:af:4d":"firetv","f0:45:da:36:e6:23":"ring-doorbell","dc:4f:22:89:fc:e7":"magichome-strip","0c:2a:69:11:01:ba":"smarter-coffee-mach","7c:49:eb:88:da:82":"xiaomi-hub","7c:49:eb:22:30:9c":"xiaomi-plug","50:32:37:b8:c7:0f":"appletv","b8:2c:a0:28:3e:6b":"honeywell-thermostat","64:16:66:2a:98:62":"nest-tstat","14:6b:9c:c6:ec:a6":"wansview-cam-wifi","78:11:dc:ec:a3:ab":"xiaomi-cleaner"}


In [33]:
dubs=['ip.flags.df', 'ip.len', 'ip.proto', 'ip.ttl', 'tls.record.length']
hexvalues=['tcp.flags']

In [34]:
# port number to port classes
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]# private port list (0-Reserved,53-DNS, 67-BOOTP server, 68-BOOTP client...)
    if port in port_list: #Is the port number in the list?
        return port_list.index(port)+1 # return the port's index number in the list (actually with index+1)
    elif 0 <= port <= 1023: # return 11 if the port number is in the range 0 :1023
        return 11
    elif  1024 <= port <= 49151 : # return 12 if the port number is in the range 1024:49151
        return 12
    elif 49152 <=port <= 65535 :# return 13 if the port number is in the range 49152:65535
        return 13
    else:# return 0 if no previous conditions are met
        return 0
    

In [35]:
# hexadecimal to decimal function
def comma_cleaner(val):
    clean=[]
    for v in val:
        if "," in str(v):
            temp=v.split(",")
            temp=temp[0]
        else:
            temp=v
        clean.append(temp)  
    return clean


In [36]:
# features with hexadecimal values
# function that holds the first value and discards the second value for repeated values
def hex2dec(val):
    clean=[]
    for v in val:
        try:
            clean.append(int(v, 16))
        except:
            clean.append(v)  
    return clean


In [41]:
path="iot-data"
pcaps=find_the_way(path,".pcap")
pcaps

['iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_17_23_20.68s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_18_38_57.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_19_39_01.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_32_13.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_51_27.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_13_35.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_35_48.64s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_57_55.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_20_02.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_42_11.66s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_17_04_20.65s.pcap',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\

In [38]:
#### pcap to csv using wireshark
#### Feature extraction main part

for  pp in tqdm(pcaps):
    name=pp.replace("\\","/")
   
    line=f"tshark -r {name} -E header=y -E separator=\"|\" -T fields -e "
    for i in feature:
        line=line+ i + " -e "
    output=name.replace(".pcap",".csv")
    line= line[:-3] + f"> {output}"
    os.system(line)

100%|██████████████████████████████████████████████████████████████████████████| 37744/37744 [2:42:30<00:00,  3.87it/s]


In [39]:
line

'tshark -r iot-data/us-vpn/zmodo-doorbell/power/2019-04-28_16_01_10.161s.pcap -E header=y -E separator="|" -T fields -e eth.src -e http.chat -e http.notification -e http.request.method -e ip.flags.df -e ip.len -e ip.proto -e ip.ttl -e ipv6.hopopts.nxt -e tcp.ack -e tcp.analysis.ack_rtt -e tcp.analysis.bytes_in_flight -e tcp.analysis.initial_rtt -e tcp.analysis.push_bytes_sent -e tcp.completeness -e tcp.dstport -e tcp.flags -e tcp.flags.push -e tcp.flags.str -e tcp.flags.syn -e tcp.hdr_len -e tcp.len -e tcp.nxtseq -e tcp.srcport -e tcp.stream -e tcp.time_delta -e tcp.time_relative -e tcp.window_size -e tcp.window_size_scalefactor -e tcp.window_size_value -e tls.record.length -e udp.checksum.status -e udp.dstport -e udp.srcport -e udp.time_delta -e udp.time_relative -e _ws.col.Protocol > iot-data/us-vpn/zmodo-doorbell/power/2019-04-28_16_01_10.161s.csv'

In [40]:
#### General corrections in CSV files (payload entropy, port class, merging of IPv4 and IPv6 properties, Labelling, etc.)
#### nan values filled with -9999
#### inf values filled with max(uint32)
files_add=find_the_way(path,".csv")
for name in tqdm(files_add):
    df=pd.read_csv(name,sep="|")
    for d in dubs:
        temp=df[d].values
        temp=comma_cleaner(temp)
        df[d]=temp
    for h in hexvalues:
        temp=df[h].values
        temp=hex2dec(temp)
        df[h]=temp

    for i in str_cols:
        temp=i.replace(".","_")
        val=df[i]
        val=val.replace(strs[temp])
        temp=hex2dec(temp)
        df[i]=val

    temp=[]


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    
    temp=[]
    for i in (df['dstport'].values):
        temp.append(port_class(i))
    df['dstport_class']=temp
    
    temp=[]
    for i in (df['srcport'].values):
        temp.append(port_class(i))
    df['srcport_class']=temp


    df['dstport'] = df['udp.dstport'].combine_first(df['udp.dstport'])
    df['srcport'] = df['udp.srcport'].combine_first(df['udp.srcport'])
    #df['ip.dst'] = df['ip.dst'].combine_first(df['ipv6.dst'])
   # df['ip.src'] = df['ip.src'].combine_first(df['ipv6.src'])
    
    df['ip.len'] = df['ip.len'].combine_first(df['ipv6.hopopts.nxt'])


    df=df.fillna(-9999)
    df["Label"]=df["eth.src"].replace(macs)
    #del df["eth.src"]
    df.to_csv(name,index=False)

100%|████████████████████████████████████████████████████████████████████████████| 37744/37744 [29:59<00:00, 20.97it/s]


In [44]:
files_add=find_the_way(path,".csv")


['iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_17_23_20.68s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_18_38_57.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-26_19_39_01.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_32_13.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_14_51_27.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_13_35.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_35_48.64s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_15_57_55.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_20_02.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_16_42_11.66s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28_17_04_20.65s.csv',
 'iot-data\\uk\\allure-speaker\\android_lan_audio_off\\2019-04-28

# Move CSV output to related folders

In [46]:
folder("csvs")

In [47]:
folders=[]
for p in pcaps:
    newname=p.replace("\\","/")
    newname=newname.split("/")
    if newname[1] not in folders:
        folders.append(newname[1])
folders

['uk', 'uk-vpn', 'us', 'us-vpn']

In [48]:
for f in folders:
    print(f)
    address=f"{path}/{f}/"
    pcaps=find_the_way(address,".csv")
    folder(address.replace("iot-data","csvs"))
    for p in tqdm(pcaps):
        newname=p.replace("\\","/")
        #newname=newname.replace("./iot-data","./pcaps")
        newname=newname[len(address):].replace("/","@")
        newname=f"{address}{newname}"
        newname=newname.replace("iot-data","csvs")
        #print(p,"\n",newname)
        os.rename(p,newname)


uk


100%|█████████████████████████████████████████████████████████████████████████████| 8023/8023 [00:09<00:00, 807.95it/s]


uk-vpn


100%|████████████████████████████████████████████████████████████████████████████| 9175/9175 [00:09<00:00, 1011.84it/s]


us


100%|██████████████████████████████████████████████████████████████████████████| 10324/10324 [00:09<00:00, 1125.62it/s]


us-vpn


100%|██████████████████████████████████████████████████████████████████████████| 10222/10222 [00:08<00:00, 1140.03it/s]


# Merge Small CSVs

### if you have split the pcap files into small sizes, you can use this section to merge their csv output

In [49]:
folders= ['uk', 'uk-vpn', 'us', 'us-vpn']
devices=[ 'appletv',
 'blink-camera',
 'blink-security-hub',
 'echodot',
 'echospot',
 'echoplus',
 'firetv',
 'google-home-mini',
 'insteon-hub',
 'lightify-hub',
 'magichome-strip',
 'nest-tstat',
 'ring-doorbell',
 'roku-tv',
 'samsungtv-wired',
 'sengled-hub',
 'smartthings-hub',
 'sousvide',
 't-philips-hub',
 'tplink-bulb',
 'tplink-plug',
 't-wemo-plug',
 'wansview-cam-wired',
 'xiaomi-hub',
 'yi-camera']

In [50]:
for f in folders:
    print(f)
    ths = open(f"./csvs/{f}.csv", "w")
    flag=1
    them=find_the_way(f"./csvs/{f}/",".csv")
    for t in tqdm(them):
        #print(t)
        with open(t, "r") as file:
            while True:
                line=file.readline()
                if line=="":break
                k=line[:-1]
                k=k.split(",")
                k=k[-1]
                if not line.startswith("eth"):
                    #print(line)
                    if "ARP" not in line:
                        if k in devices:
                            ths.write(line)
                        #print(line)
                        #break
                else:
                    if flag:
                        ths.write(line)
                        flag=0
        os.remove(t)
    ths.close()

uk


100%|█████████████████████████████████████████████████████████████████████████████| 8023/8023 [00:41<00:00, 192.81it/s]


uk-vpn


100%|█████████████████████████████████████████████████████████████████████████████| 9175/9175 [00:44<00:00, 207.63it/s]


us


100%|███████████████████████████████████████████████████████████████████████████| 10324/10324 [00:53<00:00, 191.31it/s]


us-vpn


100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:47<00:00, 215.68it/s]


In [52]:
for f in folders:

    print(f)
    df=pd.read_csv(f"./csvs/{f}.csv",usecols=["Label"])
    temp=df.groupby("Label").size()
    print(len(temp),temp)
    print("#"*120,"\n\n\n")

uk
25 Label
appletv                17977
blink-camera           83855
blink-security-hub      3352
echodot                31471
echoplus               27440
echospot               65150
firetv                 10487
google-home-mini       19209
insteon-hub               33
lightify-hub            3274
magichome-strip         1138
nest-tstat             25651
ring-doorbell         820444
roku-tv                 5876
samsungtv-wired        27148
sengled-hub             3370
smartthings-hub        21271
sousvide               15842
t-philips-hub          71515
t-wemo-plug            32546
tplink-bulb            14580
tplink-plug             5965
wansview-cam-wired    802384
xiaomi-hub              6394
yi-camera             261276
dtype: int64
######################################################################################################################## 



uk-vpn
25 Label
appletv                13775
blink-camera           92434
blink-security-hub      3863
echodot               

# Alternative method to shrink CSV
# CREATE SMALLER CSVs (1/10)

#### Using pandas on large CSV files can sometimes cause problems. In the following example I have developed a system that can sample very large files quickly and without problems.

In [4]:
folder("small")
folders= ['uk', 'uk-vpn', 'us', 'us-vpn']

In [6]:
for f in folders:
    print(f)
    df=pd.read_csv(f"./csvs/{f}.csv")#,usecols=["Label"])
    df = df.sample(n = len(df)//10) 
    df.to_csv(f"./small/{f}.csv",index=False)
    print(f,df.groupby("Label").size())
    print("__________________________________________________________________")

uk
uk Label
appletv                1797
blink-camera           8469
blink-security-hub      332
echodot                3065
echoplus               2750
echospot               6390
firetv                 1046
google-home-mini       1906
insteon-hub               3
lightify-hub            318
magichome-strip         117
nest-tstat             2550
ring-doorbell         81823
roku-tv                 604
samsungtv-wired        2691
sengled-hub             383
smartthings-hub        2159
sousvide               1560
t-philips-hub          7269
t-wemo-plug            3293
tplink-bulb            1478
tplink-plug             600
wansview-cam-wired    80410
xiaomi-hub              610
yi-camera             26141
dtype: int64
__________________________________________________________________
uk-vpn
uk-vpn Label
appletv                1392
blink-camera           9385
blink-security-hub      405
echodot                2272
echoplus               3587
echospot               8465
firetv              

### without sampling devices with a sample count below 100

In [8]:
def reduce_dataset(df, reduction_ratio):
    reduced_df = pd.DataFrame()
    for label, count in label_counts.items():
        if count >= 100:
            reduced_count = int(count * reduction_ratio)
        else:
            reduced_count = count
        label_df = df[df['Label'] == label].sample(n=reduced_count, random_state=42)
        reduced_df = pd.concat([reduced_df, label_df])
    return reduced_df

for f in folders:
    print(f)
    
    df=pd.read_csv(f"./csvs/{f}.csv")#,usecols=["Label"])
    print(f,df.groupby("Label").size())
    label_counts = df['Label'].value_counts().to_dict()
    reduction_ratio = 0.1  # Küçültme oranı
    reduced_df = reduce_dataset(df, reduction_ratio)
    reduced_df.to_csv(f"./small/{f}.csv",index=False)
    print(f,reduced_df.groupby("Label").size())
    print("__________________________________________________________________")

uk
uk Label
appletv                17977
blink-camera           83855
blink-security-hub      3352
echodot                31471
echoplus               27440
echospot               65150
firetv                 10487
google-home-mini       19209
insteon-hub               33
lightify-hub            3274
magichome-strip         1138
nest-tstat             25651
ring-doorbell         820444
roku-tv                 5876
samsungtv-wired        27148
sengled-hub             3370
smartthings-hub        21271
sousvide               15842
t-philips-hub          71515
t-wemo-plug            32546
tplink-bulb            14580
tplink-plug             5965
wansview-cam-wired    802384
xiaomi-hub              6394
yi-camera             261276
dtype: int64
uk Label
appletv                1797
blink-camera           8385
blink-security-hub      335
echodot                3147
echoplus               2744
echospot               6515
firetv                 1048
google-home-mini       1920
insteon-hub      